### Подготовка (импорт библиотек, импорт данных базы)

In [68]:
# Импортируем нужные библиотеки
import psycopg # Это Psycopg 3
import pandas

In [69]:
# Корректируем transaction.csv
transaction = pandas.read_csv('transaction.csv', sep=';')
# Удаляем некорректные значения
transaction.dropna(how='any', inplace=True)
transaction=transaction[transaction['customer_id'] != 5034]
transaction.to_csv("transaction_fixed.csv", index=None, sep=';')
transaction

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82"
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10"
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48"
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,24.06.2017,True,Approved,OHM Cycles,Standard,high,medium,"2005,66","1203,40"
19996,19997,41,127,09.11.2017,True,Approved,Solex,Road,medium,medium,"416,98","312,74"
19997,19998,87,2284,14.04.2017,True,Approved,OHM Cycles,Standard,medium,medium,"1636,9","44,71"
19998,19999,6,2764,03.07.2017,False,Approved,OHM Cycles,Standard,high,medium,"227,88","136,73"


In [70]:
# Подключаемся к БД
connection = psycopg.connect(
        host='127.0.0.1',
        user='postgres',
        password='Password123',
        dbname='test'
)

In [71]:
# Создаём таблицы
connection.execute("""
CREATE TABLE customer (
    customer_id SERIAL PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    gender VARCHAR(20),
    DOB DATE,
    job_title VARCHAR(255),
    job_industry_category VARCHAR(100),
    wealth_segment VARCHAR(50),
    deceased_indicator CHAR(1),
    owns_car CHAR(3),
    address VARCHAR(255),
    postcode VARCHAR(10),
    state VARCHAR(100),
    country VARCHAR(100),
    property_valuation INTEGER
);

CREATE TABLE transaction (
    transaction_id SERIAL PRIMARY KEY,
    product_id INTEGER,
    customer_id INTEGER REFERENCES customer(customer_id),
    transaction_date DATE,
    online_order BOOLEAN,
    order_status VARCHAR(50),
    brand VARCHAR(100),
    product_line VARCHAR(100),
    product_class VARCHAR(50),
    product_size VARCHAR(50),
    list_price DECIMAL(10, 2),
    standard_cost DECIMAL(10, 2)
);
                         """)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=127.0.0.1 user=postgres database=test) at 0x1ba3201d310>

In [72]:
# Импортируем customer
cursor = connection.cursor()
with open("customer.csv", "r") as f:
    with cursor.copy("COPY customer FROM STDIN WITH (FORMAT CSV, DELIMITER ';', HEADER)") as copy:
        while data := f.read(100):
            copy.write(data)
cursor.close()
# Проверяем
pandas.DataFrame(connection.execute("SELECT * FROM CUSTOMER").fetchall())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,None,Male,1961-10-03,None,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,n/a,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,1975-08-09,VP Product Management,Health,Mass Customer,N,No,57042 Village Green Point,4511,QLD,Australia,6
3996,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,3998,Sarene,Woolley,U,None,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
3998,3999,Patrizius,None,Male,1973-10-24,None,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7


In [73]:
# Импортируем transaction
cursor = connection.cursor()
cursor.execute("""SET DateStyle = 'German, DMY';""")
with open("transaction_fixed.csv", "r") as f:
    with cursor.copy("COPY transaction FROM STDIN WITH (FORMAT CSV, DELIMITER ';', HEADER)") as copy:
        while data := f.read(100):
            copy.write(data.replace(',','.'))
cursor.close()
pandas.DataFrame(connection.execute("SELECT * FROM TRANSACTION").fetchall())

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48
...,...,...,...,...,...,...,...,...,...,...,...,...
19437,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40
19438,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74
19439,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71
19440,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73


In [74]:
# Откат изменений
#connection.execute("ROLLBACK")

### 1. Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

In [75]:
pandas.DataFrame(connection.execute("""
    SELECT transaction_id
    FROM transaction
    WHERE order_status = 'Approved' 
        AND transaction_date BETWEEN '2017-04-01' AND '2017-04-09';
""").fetchall())

,0
0,17
1,19
2,23
3,83
4,89
...,...
509,19655
510,19853
511,19899
512,19968


### 3.Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

In [76]:
pandas.DataFrame(connection.execute("""
    SELECT DISTINCT(job_title)
    FROM customer
    WHERE job_title LIKE 'Senior%' AND
        (job_industry_category = 'IT' OR job_industry_category = 'Financial Services');
""").fetchall())

,0
0,Senior Cost Accountant
1,Senior Developer
2,Senior Editor
3,Senior Financial Analyst
4,Senior Quality Engineer
5,Senior Sales Associate


### 4. Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services.

In [77]:
pandas.DataFrame(connection.execute("""
    SELECT DISTINCT(brand)
    FROM transaction
    JOIN customer
        ON customer.customer_id = transaction.customer_id
    WHERE job_industry_category = 'Financial Services';
""").fetchall())

,0
0,Giant Bicycles
1,Norco Bicycles
2,OHM Cycles
3,Solex
4,Trek Bicycles
5,WeareA2B


### 5. Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

In [78]:
pandas.DataFrame(connection.execute("""
    SELECT DISTINCT(customer.customer_id)
    FROM transaction
    JOIN customer ON customer.customer_id = transaction.customer_id
    WHERE transaction.online_order = TRUE AND
        transaction.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
    LIMIT 10;
""").fetchall())

,0
0,1
1,2
2,3
3,5
4,6
5,7
6,8
7,11
8,12
9,13


### 6. Вывести всех клиентов, у которых нет транзакций.

In [79]:
pandas.DataFrame(connection.execute("""
    SELECT customer.customer_id
    FROM customer
    LEFT JOIN transaction
        ON customer.customer_id = transaction.customer_id
    WHERE transaction.customer_id IS NULL;
""").fetchall())

,0
0,3565
1,3535
2,3647
3,3903
4,3519
...,...
504,3811
505,3648
506,4000
507,3719


### 7. Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

In [80]:
pandas.DataFrame(connection.execute("""
    SELECT DISTINCT customer.customer_id
    FROM customer
    JOIN transaction
        ON customer.customer_id = transaction.customer_id
    WHERE transaction.standard_cost = (SELECT MAX(standard_cost) FROM transaction) AND
    customer.job_industry_category = 'IT';
""").fetchall())

,0
0,34
1,893
2,975
3,1773
4,1918
5,2913
6,3151
7,3473


### 8. Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

In [81]:
pandas.DataFrame(connection.execute("""
    SELECT DISTINCT c.customer_id
    FROM customer AS c
    JOIN transaction AS t
        ON c.customer_id = t.customer_id
    WHERE c.job_industry_category IN ('IT', 'Health')
        AND t.order_status = 'Approved'
        AND t.transaction_date BETWEEN '2017-07-07' AND '2017-07-17';
""").fetchall())

,0
0,22
1,28
2,41
3,47
4,104
...,...
107,3255
108,3288
109,3360
110,3365


In [82]:
# Отключение от БД
connection.close()